In [32]:
import pandas as pd
from source.getData import getLatestDate,subtractDays

### Get date of last 30 days.

In [43]:
today = pd.Timestamp("today").strftime("%Y-%m-%dT00:00:00.000")
last30Days = subtractDays(today,31)

In [44]:
url = f'https://healthdata.gov/resource/j8mb-icvb.json?$where=date>\"{last30Days}\"&$limit=10000&$order=date%20DESC'
url

'https://healthdata.gov/resource/j8mb-icvb.json?$where=date>"2023-04-20"&$limit=10000&$order=date%20DESC'

In [279]:
df = pd.read_json(url)
df

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported
0,AZ,Arizona,4,Region 9,Inconclusive,2023-05-18,0,28211
1,AZ,Arizona,4,Region 9,Negative,2023-05-18,801,14081648
2,AZ,Arizona,4,Region 9,Positive,2023-05-18,11,2222427
3,AR,Arkansas,5,Region 6,Inconclusive,2023-05-18,0,10479
4,AR,Arkansas,5,Region 6,Negative,2023-05-18,517,4827670
...,...,...,...,...,...,...,...,...
4234,SD,South Dakota,46,Region 8,Negative,2023-04-21,344,1128937
4235,SD,South Dakota,46,Region 8,Positive,2023-04-21,65,207271
4236,TN,Tennessee,47,Region 4,Inconclusive,2023-04-21,1,22195
4237,TN,Tennessee,47,Region 4,Negative,2023-04-21,2587,10523340


In [165]:
dc = df.loc[(df.state == 'DC')]
dc

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported
0,DC,District of Columbia,11,Region 3,Inconclusive,2023-05-18,0,14712
1,DC,District of Columbia,11,Region 3,Negative,2023-05-18,135,5561438
2,DC,District of Columbia,11,Region 3,Positive,2023-05-18,2,287782
36,DC,District of Columbia,11,Region 3,Inconclusive,2023-05-17,0,14712
37,DC,District of Columbia,11,Region 3,Negative,2023-05-17,391,5561303
...,...,...,...,...,...,...,...,...
3832,DC,District of Columbia,11,Region 3,Negative,2023-04-22,300,5551602
3833,DC,District of Columbia,11,Region 3,Positive,2023-04-22,16,287472
3990,DC,District of Columbia,11,Region 3,Inconclusive,2023-04-21,0,14709
3991,DC,District of Columbia,11,Region 3,Negative,2023-04-21,465,5551302


In [275]:
#trails with one state
dc_NewResultsReported = dc['new_results_reported'].sum()
print(f'dc_totalNewCases = {dc_newCases}')

dc_pCases = dc.loc[dc.overall_outcome == 'Positive']['new_results_reported'].sum()
print(f'dc_pCases = {dc_pCases}')

dc_pRate = (dc_pCases/dc_NewResultsReported)*100
print(f'dc_pRate = {dc_pRate}')

#trails on creating a new datafram with just one (or two) state's data
dc_data = [[dc.loc[0]['state'],dc.loc[0]['state_name'],dc_NewResultsReported,dc_pCases,dc_pRate],
           ['DC',dc.loc[0]['state_name'],dc_NewResultsReported,dc_pCases,dc_pRate]]
print(type(dc_data))
print(dc_data)
dc_columuns=['state','state_name','dc_NewResultsReported','positive_cases','positivity_rate']
dc_df = pd.DataFrame(dc_data,columns=dc_columuns)
dc_df

dc_totalNewCases = 10946
dc_pCases = 342
dc_pRate = 3.124429015165357
<class 'list'>
[['DC', 'District of Columbia', 10946, 342, 3.124429015165357], ['DC', 'District of Columbia', 10946, 342, 3.124429015165357]]


,state,state_name,dc_NewResultsReported,positive_cases,positivity_rate
0,DC,District of Columbia,10946,342,3.124429
1,DC,District of Columbia,10946,342,3.124429


In [179]:
df.state.unique()

array(['DC', 'IL', 'LA', 'MI', 'NY', 'ND', 'OH', 'SD', 'WI', 'AZ', 'AR',
       'DE', 'GA', 'ID', 'KS', 'KY', 'ME', 'MA', 'MN', 'MS', 'MT', 'NE',
       'NH', 'NC', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'UT', 'VT', 'VA',
       'WY', 'AK', 'GU', 'MD', 'TX', 'WV', 'IN', 'NV', 'TN', 'WA', 'HI',
       'VI', 'CO', 'NJ', 'FL', 'CT', 'MO', 'NM', 'AL', 'CA'], dtype=object)

In [183]:
len(df.state.unique())

53

In [280]:
state_columns=['state','state_name','new_results_reported','positive_cases','positivity_rate']
state_data = []
for current_state in df.state.unique():
    state = df.loc[(df.state == current_state)]        
    state_name = state.iloc[0]['state_name']    
    state_NewResultsReported = state['new_results_reported'].sum()    
    state_pCases = state.loc[state.overall_outcome == 'Positive']['new_results_reported'].sum()
    state_pRate = (state_pCases/state_NewResultsReported)*100
    current_state_data = [current_state,state_name,state_NewResultsReported,state_pCases,state_pRate]
    state_data.append(current_state_data)
state_df = pd.DataFrame(state_data, columns=state_columns)
state_df

,state,state_name,new_results_reported,positive_cases,positivity_rate
0,AZ,Arizona,60795,4744,7.803273
1,AR,Arkansas,19147,1286,6.716457
2,DE,Delaware,10440,338,3.237548
3,DC,District of Columbia,11336,349,3.078687
4,GA,Georgia,107303,5443,5.072552
5,ID,Idaho,8769,819,9.339719
6,IL,Illinois,339131,10655,3.141854
7,KS,Kansas,18119,1217,6.716706
8,KY,Kentucky,49183,3038,6.176931
9,LA,Louisiana,53047,2321,4.375365


In [277]:
state_df.sort_values(by='positivity_rate',ascending=False).head(10)

,state,state_name,new_results_reported,positive_cases,positivity_rate
30,VI,U.S. Virgin Islands,177,50,28.248588
50,MO,Missouri,20913,3865,18.481327
25,PR,Puerto Rico,21679,3972,18.321878
28,SD,South Dakota,9744,1673,17.169540
37,GU,Guam,1604,256,15.960100
42,WA,Washington,34762,4922,14.159139
35,WY,Wyoming,5006,581,11.606073
51,NM,New Mexico,17450,1944,11.140401
45,HI,Hawaii,19391,1994,10.283121
44,NV,Nevada,22114,2227,10.070544


## All the above in one block of code

In [1]:
import pandas as pd
from source.getData import subtractDays

# Keeping the date dynamic:
today = pd.Timestamp("today").strftime("%Y-%m-%d")
# going back 30 days (31 days for the url $where=date> to work as intended to grab the data for last 30 days)
last30Days = subtractDays(today,31)

url = f'https://healthdata.gov/resource/j8mb-icvb.json?$where=date>\"{last30Days}\"&$limit=10000&$order=date%20DESC'
df = pd.read_json(url)

# Iterate over the data for each state and calculate positivity_rate
state_data = []
for current_state in df.state.unique():
    state = df.loc[(df.state == current_state)]
    state_name = state.iloc[0]['state_name']
    state_NewResultsReported = state['new_results_reported'].sum()
    state_pCases = state.loc[state.overall_outcome == 'Positive']['new_results_reported'].sum()
    state_pRate = (state_pCases/state_NewResultsReported)*100
    current_state_data = [current_state,state_name,state_NewResultsReported,state_pCases,state_pRate]
    state_data.append(current_state_data)

# Create a new data frame with the above calculated values
state_columns=['state','state_name','new_results_reported','positive_cases','positivity_rate']
state_df = pd.DataFrame(state_data, columns=state_columns)

# The 10 states with the highest test positivity rate (positive tests / tests performed) 
# for tests performed in the last 30 days.
state_df.sort_values(by='positivity_rate',ascending=False).head(10)

,state,state_name,new_results_reported,positive_cases,positivity_rate
30,VI,U.S. Virgin Islands,177,50,28.248588
50,MO,Missouri,20913,3865,18.481327
25,PR,Puerto Rico,21679,3972,18.321878
28,SD,South Dakota,9744,1673,17.169540
37,GU,Guam,1604,256,15.960100
42,WA,Washington,34762,4922,14.159139
35,WY,Wyoming,5006,581,11.606073
51,NM,New Mexico,17450,1944,11.140401
45,HI,Hawaii,19391,1994,10.283121
44,NV,Nevada,22114,2227,10.070544
